$\textbf{IMPORTS}$

In [8]:
import requests
import numpy as np
import csv
import json
import time
import pprint
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{LOAD ALL PAGES}$, BOLIGA.DK

In [2]:
#1.Load pages boliga.dk
links = []
for page in range(1,148): #check the number of pages in link before running this code!!
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}&sort=daysForSale-a'
    links.append(url_boliga)

$\textbf{SCRAPE DATA FROM EACH PAGE}$, BOLIGA.DK

In [3]:
#Combinale all code above in a loop to scrape all pages
add = []
p = []
zipc = []
m_2 = []
rooms_ = []

for i in links:
    response = requests.get(i, headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    soup = BeautifulSoup(response.content,"xml")
    
    #append address to list
    list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
    address = [col.text.split(",",1)[0] for col in list_address]
    add.append(address)
    #convert list of lists to ONE LIST
    address_final = []
    for sublist in add:
        for item in sublist:
            address_final.append(item)
    
    #append price to list
    list_price = soup.find_all("div" , class_ ="price m-0 d-flex align-items-center")
    price = [col.text for col in list_price]
    p.append(price)
    #convert list of lists to ONE LIST
    price_final = []
    for sublist in p:
        for item in sublist:
            price_final.append(item)
    
    #append zipcode to list
    list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
    zipcode = [col.text for col in list_zipcode]
    zipc.append(zipcode)
     #convert list of lists to ONE LIST
    zipcode_final = []
    for sublist in zipc:
        for item in sublist:
            zipcode_final.append(item)
    
    #append m2 to list
    list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
    m2 = [col.text.split() for col in list_m2]
    #the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
    m2 = [element[0] for element in m2]
    del m2[0] #delete first element in m2 in order to get all m2 in next code
    m2 = m2[::6] #we now keep every 6th element, which is the size of every address
    m_2.append(m2)
    #convert list of lists to ONE LIST
    m2_final = []
    for sublist in m_2:
        for item in sublist:
            m2_final.append(item)
    
    #append room to list
    rooms = [col.text.split() for col in list_m2]
    rooms = [elements[1:] for elements in rooms]
    rooms = rooms[::6] #we now keep every 6th element, which is no. of rooms
    rooms_.append(rooms)
    #convert list of lists to ONE LIST
    rooms_fin = []
    for sublist in rooms_:
        for item in sublist:
            rooms_fin.append(item)
    rooms_final = []
    for sublist in rooms_fin:
        for item in sublist:
            rooms_final.append(item)
    #replace "værelse" with "1"
    for i in range(len(rooms_final)):
        if rooms_final[i] == "værelse":
            rooms_final[i] = "1"
    time.sleep(0.5) #Sleep for 0.5 seconds


$\textbf{COMBINE ALL CODE ABOVE INTO ONE DATAFRAME}$

In [4]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"Pris": price_final, "address": address_final, "zipcode": zipcode_final, "m2": m2_final, "rooms": rooms_final}
df_data = pd.DataFrame(df_addzip)
df_data

,Pris,address,zipcode,m2,rooms
0,265.000kr.,Nørmarkvej 29 1 12,7600 Struer,57,2
1,475.000kr.,Skelvangsvej 105,8920 Randers NV,67,2
2,495.000kr.,Lyksborgvej 38,7500 Holstebro,43,2
3,495.000kr.,Nyvangsvej 18,8900 Randers C,50,2
4,525.000kr.,Dalgas Alle 109,7800 Skive,47,1
...,...,...,...,...,...
7236,3.225.000kr.,Bydammen 33,2750 Ballerup,88,3
7237,3.750.000kr.,Bydammen 33,2750 Ballerup,107,3
7238,2.550.000kr.,Bydammen 23,2750 Ballerup,71,2
7239,2.695.000kr.,Bydammen 21,2750 Ballerup,75,2


$\textbf{GET  DISTANCES}, \ dataforsyningen.dk, \ hvorlangterder.dk$

In [5]:
#1. Load pages
Address_zipcode = df_data["address"] + " " + df_data["zipcode"]
add_zip = []
for i in Address_zipcode:
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={i}&struktur=mini", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    res_j = response_dist.json()
    add_zip.append(res_j)

Before continuing, locate missing values due to typing error in column address in df_data

In [6]:
#2. Locate missing values
df_add_zip = pd.DataFrame(add_zip)
index = df_add_zip[df_add_zip[0].isnull()][0]

#3. Store index numbers in order to del the corresponding indexes in df_data
index_list = list(index.index.values)

#4. Delete corresponding rows in df_data
for i in index_list:
    df_add_zip.drop(i, inplace=True)

#5. reset_index keeps failing, so here comes another index-method    
df_add_zip["new_index"] = range(len(df_add_zip))
df_add_zip.set_index("new_index", inplace=True)

#vi mister 7254 - 7114 observationer efter fjernelse af missing values!!

Drop missing values in df_data

In [7]:
#6. Delete corresponding rows in df_data
for i in index_list:
    df_data.drop(i, inplace=True)

#reset_index keeps failing, so here comes another index-method    
df_data["new_index"] = range(len(df_data))
df_data.set_index("new_index", inplace=True)

#vi mister 7254 - 7114 observationer efter fjernelse af missing values!!

In [11]:
#7. Get longitude, x, and latitudes, y from address i
x = []
y = []

for i in range(len(df_add_zip)):
    xx = df_add_zip[0][i]["x"]
    x.append(xx)

for i in range(len(df_add_zip)):
    yy = df_add_zip[0][i]["y"]
    y.append(yy)

In [27]:
#8. Add coordinates to df_data
df_data["x"] = x
df_data["y"] = y
xy = df_data["y"].apply(str) +","+ df_data["x"].apply(str)

In [13]:
#3 Get distance to nearest daycare
daycare_distance = []

for i in xy:
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=daycare&decodepolyline')
    afstand_daycare_json.json()
    key = afstand_daycare_json.json()['daycare']
    key = key['routedmeters']
    daycare_distance.append(key)

In [15]:
regionaltog_distance = []

for i in xy:
    afstand_regionaltog_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=train&decodepolyline')
    afstand_regionaltog_json.json()
    key = afstand_regionaltog_json.json()['train']
    key = key['routedmeters']
    regionaltog_distance.append(key)

In [16]:
school_distance = []

for i in xy:
    afstand_school_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=school&decodepolyline')
    afstand_school_json.json()
    key = afstand_school_json.json()['school']
    key = key['routedmeters']
    school_distance.append(key)

In [28]:
supermarket_distance = []

for i in xy:
    afstand_supermarket_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=supermarket&decodepolyline')
    afstand_supermarket_json.json()
    key = afstand_supermarket_json.json()['supermarket']
    key = key['routedmeters']
    supermarket_distance.append(key)

In [29]:
hospital_distance = []

for i in xy:
    hospital_distance_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=hospital&decodepolyline')
    hospital_distance_json.json()
    key = hospital_distance_json.json()['hospital']
    key = key['routedmeters']
    hospital_distance.append(key)

$\textbf{FINAL DATASET}$

In [36]:
#4 Add coordinates of addresses in dataframe
df_data["daycare_distance"] = daycare_distance
df_data["train_distance"] = regionaltog_distance
df_data["school_distance"] = school_distance
df_data["supermarket_distance"] = supermarket_distance
df_data["hospital_distance"] = hospital_distance

In [12]:
df_data = pd.read_csv(Path.cwd() / 'v4dist.csv') 

FileNotFoundError: [Errno 2] No such file or directory: '/home/jonasmorsing/SDS/v4dist.csv'

In [11]:
df_data

NameError: name 'df_data' is not defined

In [27]:
response_reg = requests.get(f"https://api.dataforsyningen.dk/regioner", headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
res_reg = response_reg.json()


In [28]:
regions = []

for i in range(0,5):
    rg = res_reg[i]["bbox"]
    regions.append(rg)
    print(regions)

[[8.18951662, 56.53454667, 11.22599137, 57.76025478]]
[[8.18951662, 56.53454667, 11.22599137, 57.76025478], [8.07887623, 55.64437916, 11.66419061, 56.84325702]]
[[8.18951662, 56.53454667, 11.22599137, 57.76025478], [8.07887623, 55.64437916, 11.66419061, 56.84325702], [8.06320315, 54.71828064, 10.99555236, 55.95324986]]
[[8.18951662, 56.53454667, 11.22599137, 57.76025478], [8.07887623, 55.64437916, 11.66419061, 56.84325702], [8.06320315, 54.71828064, 10.99555236, 55.95324986], [11.60211644, 54.98354657, 15.31831444, 56.20520188]]
[[8.18951662, 56.53454667, 11.22599137, 57.76025478], [8.07887623, 55.64437916, 11.66419061, 56.84325702], [8.06320315, 54.71828064, 10.99555236, 55.95324986], [11.60211644, 54.98354657, 15.31831444, 56.20520188], [10.8148054, 54.54440636, 12.64551576, 56.0173058]]


In [29]:
regions

[[8.18951662, 56.53454667, 11.22599137, 57.76025478],
 [8.07887623, 55.64437916, 11.66419061, 56.84325702],
 [8.06320315, 54.71828064, 10.99555236, 55.95324986],
 [11.60211644, 54.98354657, 15.31831444, 56.20520188],
 [10.8148054, 54.54440636, 12.64551576, 56.0173058]]

In [59]:
def region(df_data):
    
    if (8.18951662 < df_data['x'] < 11.22599137) and (56.53454667 < df_data['y'] < 57.76025478):  
        return 'Nordjylland'
    elif (8.07887623 <  df_data['x'] < 11.66419061) and (55.64437916 < df_data['y'] < 56.84325702): 
        return 'Midtjylland'
    elif (8.06320315 < df_data['x'] < 10.99555236) and (54.71828064 < df_data['y'] < 55.95324986):  
        return 'Syddanmark'
    elif (11.60211644 < df_data['x'] < 15.31831444) and (54.98354657 < df_data['y'] < 56.20520188):  
        return 'Hovedstaden'
    elif (10.8148054 < df_data['x'] < 12.64551576) and (54.54440636 < df_data['y'] < 56.0173058):  
        return 'Sjælland'
df_data['Region'] = df_data.apply(region, axis=1)
display(df_data)

,new_index,Pris,address,zipcode,m2,rooms,x,y,daycare_distance,train_distance,school_distance,supermarket_distance,hospital_distance,Region
0,0,265.000kr.,Nørmarkvej 29 1 12,7600 Struer,57,2,8.595832,56.482711,464.56,1594.73,1169.06,497.99,15116.95,Midtjylland
1,1,475.000kr.,Skelvangsvej 105,8920 Randers NV,67,2,10.013025,56.485227,490.10,3649.52,1510.22,1165.91,4289.22,Midtjylland
2,2,495.000kr.,Lyksborgvej 38,7500 Holstebro,43,2,8.612840,56.371653,259.20,1427.94,477.51,723.78,1964.17,Midtjylland
3,3,495.000kr.,Nyvangsvej 18,8900 Randers C,50,2,10.027365,56.466636,234.05,1197.57,358.13,404.26,2083.77,Midtjylland
4,4,525.000kr.,Dalgas Alle 109,7800 Skive,47,1,9.021491,56.546826,892.49,2126.17,892.49,1217.77,3977.31,Nordjylland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7097,7097,3.225.000kr.,Bydammen 33,2750 Ballerup,88,3,12.358815,55.730680,810.92,11032.11,713.88,227.25,7111.65,Hovedstaden
7098,7098,3.750.000kr.,Bydammen 33,2750 Ballerup,107,3,12.358815,55.730680,810.92,11032.11,713.88,227.25,7111.65,Hovedstaden
7099,7099,2.550.000kr.,Bydammen 23,2750 Ballerup,71,2,12.359918,55.731344,735.14,11938.71,638.10,97.27,7118.35,Hovedstaden
7100,7100,2.695.000kr.,Bydammen 21,2750 Ballerup,75,2,12.359697,55.731365,722.72,11954.29,625.68,112.85,7133.93,Hovedstaden


In [1]:
df_data.to_csv('v5With_regions.csv', index=False)

NameError: name 'df_data' is not defined